In [1]:
from salishsea_tools import viz_tools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import xarray as xr
import cmocean.cm as cm
import os
import re

In [2]:
fnames = os.listdir('01mar23_31mar23')

FileNotFoundError: [Errno 2] No such file or directory: '01mar23_31mar23'

In [ ]:
grid_Ts = []
rivr_Ts = []
# grid_Us = []
# grid_Vs = []

for file in fnames:
    print(file)
    if re.search(r'.*grid_T*', file):
        grid_Ts = grid_Ts + [xr.open_dataset('01mar23_31mar23/' + file)]

    elif re.search(r'.*rivr_T*', file):
        rivr_Ts = rivr_Ts + [xr.open_dataset('01mar23_31mar23/' + file)]

    # elif re.search(r'.*grid_U*', file):
    #     grid_Us = grid_Us + [xr.open_dataset('01mar23_31mar23/' + file)]

    # elif re.search(r'.*grid_V*', file):
    #     grid_Vs = grid_Vs + [xr.open_dataset('01mar23_31mar23/' + file)]

    else:
        print('no file match')

: 

In [ ]:
len(grid_Ts)

: 

In [ ]:
## import the entire dataset
# phys_rivers = xr.open_dataset('01mar23_05mar23_prop/SalishSea_1h_20230301_20230305_grid_T_20230305-20230305.nc')
# fw_rivers = xr.open_dataset('01mar23_05mar23_prop/SalishSea_1h_20230301_20230305_rivr_T_20230305-20230305.nc')
# uvels = xr.open_dataset('01mar23_05mar23_prop/SalishSea_1h_20230301_20230305_grid_U_20230305-20230305.nc')
# vvels = xr.open_dataset('01mar23_05mar23_prop/SalishSea_1h_20230301_20230305_grid_V_20230305-20230305.nc')

phys_rivers = xr.open_dataset('01apr23_30apr23/SalishSea_1d_20230401_20230501_grid_T_20230401-20230401.nc')
fw_rivers = xr.open_dataset('01apr23_30apr23/SalishSea_1d_20230401_20230501_rivr_T_20230401-20230401.nc')

# phys_rivers = xr.open_dataset('01mar23_31mar23/SalishSea_1d_20230301_20230331_grid_T_20230331-20230331.nc')
# fw_rivers = xr.open_dataset('01mar23_31mar23/SalishSea_1d_20230301_20230331_rivr_T_20230331-20230331.nc')
# uvels = xr.open_dataset('01mar23_05mar23_prop/SalishSea_1h_20230301_20230305_grid_U_20230305-20230305.nc')
# vvels = xr.open_dataset('01mar23_05mar23_prop/SalishSea_1h_20230301_20230305_grid_V_20230305-20230305.nc')

# phys_rivers = xr.open_dataset('01mar23_31mar23/SalishSea_1d_20230301_20230331_grid_T_20230315-20230315.nc')
# fw_rivers = xr.open_dataset('01mar23_31mar23/SalishSea_1d_20230301_20230331_rivr_T_20230315-20230315.nc')

: 

In [ ]:
phys_old = xr.open_dataset('../../../../../../../../results2/SalishSea/nowcast-green.202111/01mar23/SalishSea_1h_20230301_20230301_grid_T.nc')

: 

In [ ]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]

: 

## Surface salinity and temperature

In [ ]:
cmap = cm.haline
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
salinity = np.ma.masked_array(phys_rivers.vosaline[0, 0], mask=tmask[0])
colours = ax.pcolormesh(salinity, cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('Salinity (g/kg)')

: 

In [ ]:
cmap = cm.thermal
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
temp = np.ma.masked_array(phys_rivers.votemper[0, 0], mask=tmask[0])
colours = ax.pcolormesh(temp, vmin=0, vmax=10, cmap=cmap)  #TODO: survey an entire year to pick the best vmax in the Salish Sea
cb = fig.colorbar(colours, ax=ax)
cb.set_label(r'Temperature ($^o C$)')

: 

## Check salinity and temp against the base run

In [ ]:
tstep = 0

cmap = mpl.colormaps['RdBu']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
salinity = np.ma.masked_array(phys_old.vosaline[tstep, 0] - phys_rivers.vosaline[tstep, 0], mask=tmask[0])
colours = ax.pcolormesh(salinity, cmap=cmap, vmin=-1.5,vmax=1.5)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('Salinity 1D Run - Salinity 3D Run (g/kg)')
ax.set_title('Timestep = {}'.format(str(tstep)))

: 

In [ ]:
tstep = 0

cmap = mpl.colormaps['RdBu']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
temperature = np.ma.masked_array(phys_old.votemper[tstep, 0] - phys_rivers.votemper[tstep, 0], mask=tmask[0])
colours = ax.pcolormesh(temperature, cmap=cmap, vmin=-1, vmax=1)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('Temp 1D Run - Temp 3D Run')
ax.set_title('Timestep = {}'.format(str(tstep)))

: 

## Rivers

In [ ]:
import salishsea_tools.river_202108 as rivers

: 

In [ ]:
def river_box(w_shed, r_call, box_side):  #TODO: change this to include the other river that falls off the edge
    river_idx = [rivers.prop_dict[w_shed][r_call]['i'], rivers.prop_dict[w_shed][r_call]['j']]
    river_range = [river_idx[0] - box_side, river_idx[0] + box_side, river_idx[1] - box_side, river_idx[1] + box_side]

    if (river_range[0] < 0):
        river_range[0] = 0
        river_range[1] = box_side*2

    if (898 - river_range[1] < 0):
        river_range[0] = 898-box_side*2
        river_range[1] = 898

    return river_range

: 

In [ ]:
box_size = 30
# order (CW starting under the Fraser): Nooksack, Skagit, Snohomish, Nisqually, Elwha, Cowichan, Nanaimo, Puntledge, Salmon, Homathko, Squamish
Nooksack = river_box('fraser', 'Nooksack', box_size)
Skagit = river_box('skagit', 'Skagit1', box_size)
Snohomish = river_box('skagit', 'SnohomishAllenQuilceda', box_size)
Nisqually = river_box('puget', 'NisquallyMcAllister', box_size)
Elwha = river_box('jdf', 'Elwha', box_size)
Cowichan = river_box('evi_s', 'Cowichan1', box_size)
Nanaimo = river_box('evi_s', 'Nanaimo1', box_size)
Puntledge = river_box('evi_s', 'Puntledge', box_size)
Salmon = river_box('evi_n', 'SalmonSayward', box_size)
Homathko = river_box('bute', 'Homathko', box_size)
Squamish = river_box('howe', 'Squamish', box_size)

: 

In [ ]:
gridX = np.arange(Nooksack[0],Nooksack[1])
gridY = np.arange(Nooksack[2],Nooksack[3])

: 

# Visualizations of dye

In [ ]:
fudge = .0001 # need to add very small number here because we can't do log of zero without weird effects in the masking

: 

In [ ]:
np.nansum(fw_rivers.nooksack_river[tstep, 0].to_numpy())

: 

In [ ]:
fw_rivers.skagit_river

: 

## View all dye in the domain

In [ ]:
tstep = 0

subset_rivers = fw_rivers.nooksack_river[tstep, 0] + fw_rivers.skagit_river[tstep, 0] + fw_rivers.snohomish_river[tstep, 0] + fw_rivers.nisqually_river[tstep, 0] + fw_rivers.elwha_river[tstep, 0] + \
    fw_rivers.cowichan_river[tstep, 0] + fw_rivers.nanaimo_river[tstep, 0] + fw_rivers.puntledge_river[tstep, 0] + fw_rivers.salmon_river[tstep, 0] + fw_rivers.homathko_river[tstep, 0] + \
    fw_rivers.squamish_river[tstep, 0]

freshwater = np.ma.masked_array(subset_rivers, mask=tmask[0])

X = np.arange(0,398)
Y = np.arange(0,898)
Z = freshwater + fudge

cmap = mpl.colormaps['Reds']
cmap.set_bad('grey')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('dye')
ax.set_title('River Subset, Timestep = {}'.format(str(tstep)))

: 

In [ ]:
tstep = 0

subset_rivers = fw_rivers.nooksack_river[tstep, 0]

freshwater = np.ma.masked_array(subset_rivers, mask=tmask[0])

X = np.arange(0,398)
Y = np.arange(0,898)
Z = freshwater + fudge

cmap = mpl.colormaps['Reds']
cmap.set_bad('grey')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('dye')
ax.set_title('nooksack River, Timestep = {}'.format(str(tstep)))

: 

In [ ]:
tstep = 0

freshwater = np.ma.masked_array(fw_rivers.other_rivers[tstep, 0], mask=tmask[0])

X = np.arange(0,398)
Y = np.arange(0,898)
Z = freshwater + fudge

cmap = mpl.colormaps['Reds']
cmap.set_bad('grey')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('dye')
ax.set_title('Other Rivers, Timestep = {}'.format(str(tstep)))

: 

In [ ]:
tstep = 0

freshwater = np.ma.masked_array(fw_rivers.fraser_river[tstep, 0], mask=tmask[0])

X = np.arange(0,398)
Y = np.arange(0,898)
Z = freshwater + fudge

cmap = mpl.colormaps['Reds']
cmap.set_bad('grey')
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('dye')
ax.set_title('Fraser River, Timestep = {}'.format(str(tstep)))

: 

## Surface plan views

In [ ]:
tstep = 0
freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep, 0], mask=tmask[0])

X = gridX
Y = gridY
Z = freshwater[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]] + fudge

cmap = mpl.colormaps['Reds']
cmap.set_bad('grey')
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('freshwater nooksack')
ax.set_title('Timestep = {}'.format(str(tstep)))

: 

In [ ]:
# tstep = 0

# freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep, 0], mask=tmask[0])
# X = fw_rivers.nav_lon[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]]
# Y = fw_rivers.nav_lat[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]]
# Z = freshwater[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]] + fudge  # need to add very small number here because we can't do log of zero without weird effects in the masking

# cmap = mpl.colormaps['Reds']
# cmap.set_bad('gray')
# fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
# cb = fig.colorbar(colours, ax=ax)
# cb.set_label('freshwater nooksack')
# ax.set_title('Timestep = {}'.format(str(tstep)))

: 

In [ ]:
fw_rivers.nooksack_river

: 

In [ ]:
fw_rivers.nooksack_river[0][:, :, rivers.prop_dict['fraser']['Nooksack']['j']]

: 

In [ ]:
tmask[:, :, rivers.prop_dict['fraser']['Nooksack']['j']]

: 

In [ ]:
Z.shape

: 

## Side slice views

In [ ]:
# vertical slice

tstep = 0

cmap = mpl.colormaps['Reds']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep][:, :, rivers.prop_dict['fraser']['Nooksack']['j']], mask=tmask[:, :, rivers.prop_dict['fraser']['Nooksack']['j']])

X = fw_rivers.nav_lat[Nooksack[0]:Nooksack[1], rivers.prop_dict['fraser']['Nooksack']['j']+1]
Y = fw_rivers.deptht
Z = freshwater[:, Nooksack[0]:Nooksack[1]] + fudge

colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)

cb = fig.colorbar(colours, ax=ax)
cb.set_label('freshwater nooksack')
ax.set_title('Timestep = {}'.format(str(tstep)))
ax.set_ylim(0, 20)

plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

: 

In [ ]:
# horizontal slice

tstep = 0

cmap = mpl.colormaps['Reds']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep, :, rivers.prop_dict['fraser']['Nooksack']['i'], :], mask=tmask[:, rivers.prop_dict['fraser']['Nooksack']['i'], :])

X = fw_rivers.nav_lon[rivers.prop_dict['fraser']['Nooksack']['i']+1, Nooksack[2]:Nooksack[3]]
Y = fw_rivers.deptht
Z = freshwater[:, Nooksack[2]:Nooksack[3]] + fudge

colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)

cb = fig.colorbar(colours, ax=ax)
cb.set_label('freshwater nooksack')
ax.set_title('Timestep = {}'.format(str(tstep)))
ax.set_ylim(0, 20)

plt.gca().invert_yaxis()

: 

# Sum views

In [ ]:
tstep = 0

cmap = mpl.colormaps['Reds']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep, :], mask=tmask[:])
freshwater = freshwater.sum(axis=0)

X = gridX
Y = gridY
Z = freshwater[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]] + fudge

colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('freshwater nooksack')
ax.set_title('Plan Sum \nTimestep = {}'.format(str(tstep)))

: 

In [ ]:
# vertical slice sum
sum_size = 30
tstep = 0

cmap = mpl.colormaps['Reds']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep, :, :, :], mask=tmask[:, :, :])

X = fw_rivers.nav_lat[Nooksack[0]:Nooksack[1], rivers.prop_dict['fraser']['Nooksack']['j']+1]
Y = fw_rivers.deptht
Z = np.ma.masked_array(freshwater[:, Nooksack[0]:Nooksack[1], rivers.prop_dict['fraser']['Nooksack']['j']-sum_size:rivers.prop_dict['fraser']['Nooksack']['j']+sum_size].sum(axis=2), mask=tmask[:, Nooksack[0]:Nooksack[1], rivers.prop_dict['fraser']['Nooksack']['j']])

colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)

cb = fig.colorbar(colours, ax=ax)
cb.set_label('freshwater nooksack')
ax.set_title('Slice Sum \nTimestep = {}'.format(str(tstep)))
ax.set_ylim(0, 20)

plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

: 

In [ ]:
# horizontal slice sum
sum_size = 30
tstep = 0

cmap = mpl.colormaps['Reds']
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
freshwater = np.ma.masked_array(fw_rivers.nooksack_river[tstep, :, :, :], mask=tmask[:, :, :])

X = fw_rivers.nav_lon[rivers.prop_dict['fraser']['Nooksack']['i']+1, Nooksack[2]:Nooksack[3]]
Y = fw_rivers.deptht
Z = np.ma.masked_array(freshwater[:, rivers.prop_dict['fraser']['Nooksack']['i']-sum_size:rivers.prop_dict['fraser']['Nooksack']['i']+sum_size, Nooksack[2]:Nooksack[3]].sum(axis=1), mask=tmask[:, rivers.prop_dict['fraser']['Nooksack']['i'], Nooksack[2]:Nooksack[3]])

colours = ax.pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=fudge), cmap=cmap)

cb = fig.colorbar(colours, ax=ax)
cb.set_label('freshwater nooksack')
ax.set_title('Slice Sum \nTimestep = {}'.format(str(tstep)))
ax.set_ylim(0, 20)

plt.gca().invert_yaxis()

: 

# Surface velocity

In [ ]:
zoom_fac = 13
tstep = 0


vel_mag = np.sqrt(vvels.vomecrty[tstep, 0]**2 + vvels.vomecrty[tstep, 0]**2)

cmap = cm.speed
cmap.set_bad('gray')
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

u_vels = np.ma.masked_array(uvels.vozocrtx[tstep, 0], mask=tmask[0])[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]]
v_vels = np.ma.masked_array(vvels.vomecrty[tstep, 0], mask=tmask[0])[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]]
freshwater = np.ma.masked_array(vel_mag, mask=tmask[0])

X = gridX
Y = gridY
Z = freshwater[Nooksack[0]:Nooksack[1], Nooksack[2]:Nooksack[3]] + fudge

colours = ax.pcolormesh(X, Y, Z, vmin=0, vmax=1, cmap=cmap)
cb = fig.colorbar(colours, ax=ax)
cb.set_label('velocity magnitude (m/s)')

X, Y = np.meshgrid(X, Y)

ax.quiver(X, Y, u_vels, v_vels, scale=5)
ax.set_title('Velocity \nTimestep = {}'.format(str(tstep)))
ax.set_xlim(Nooksack[0]+zoom_fac, Nooksack[1]-zoom_fac)
ax.set_ylim(Nooksack[2]+zoom_fac, Nooksack[3]-zoom_fac)

: 

In [ ]:
#TODO: likely in another script, should i have one that has each individual water? if not, i need to show that they are far enough away that they don't really interact. when does water no longer "belong" to each river?

: 